In [2]:
import random
n=10
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [3]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [4]:
import itertools
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    return res

In [5]:
import heapq
def mst(ps):
    n = len(ps)
    #prim
    mst = {v:[] for v in range(n)} # MST の隣接リスト
    X = set()
    heap = []
    for v in range(1,n):
        heapq.heappush(heap,(t_cost(ps[0],ps[v]),0,v))
    while len(heap)>0:
        (d,u,v)=heapq.heappop(heap)
        if v not in X:
            X.add(v)
            mst[u].append(v)
            mst[v].append(u)
            for w in range(n):
                if w not in X: heapq.heappush(heap,(t_cost(ps[v],ps[w]),v,w))
    #dfs
    res = []
    stack = [0]
    visited = [False]*n
    while len(stack)>0:
        v = stack.pop()
        if visited[v]: continue
        visited[v]=True
        res.append(v)
        for u in mst[v]: stack.append(u)
    return res 

In [6]:
def dp(ps):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    return route[(0,frozenset(range(n)))] 

In [7]:
import itertools
import numpy
def greedy_core(tsp,depot):#depotはindex指定ではない
    label={}
    operation={}
    flyop={}
    driveop={}
    tsptotal=0
    for i in tsp:
        label[i]='simple'
        tsptotal=tsptotal+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    n=len(label)
    tspdtotal=tsptotal
    while 'simple' in label.values():
        ms=[]
        for i in range(n):
            if label[tsp[i]]!='simple' or tsp[i]==depot:
                ms.append(-float('inf'))
            else:
                mfsi=t_cost(V[tsp[i-1]],V[tsp[i]])+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])-max(d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]]),t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]]))
                ms.append(mfsi)
        for i in range(n):
            if label[tsp[i-1]]=='combined' and tsp[(i-1)]!=depot:
                if label[tsp[i]]=='simple':
                    j=i-2
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j-1)]]=='combined':
                            pls=t_cost(V[tsp[i-1]],V[tsp[i]])+operation[(tsp[dronenode],tsp[j-1],tsp[i-1])]-max(d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i]]),driveop[(tsp[dronenode],tsp[j-1],tsp[i-1])]+t_cost(V[tsp[i-1]],V[tsp[i]]))
                            ms.append(pls)
                            break
                        j=j-1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(n):
            if label[tsp[(i+1)%n]]=='combined' and tsp[(i+1)%n]!=depot:
                if label[tsp[i]]=='simple':
                    j=(i+2)%n
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j+1)%n]]=='combined':
                            prs=t_cost(V[tsp[i]],V[tsp[(i+1)%n]])+operation[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]-max(d_cost(V[tsp[i]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]]),driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i]],V[tsp[(i+1)%n]]))
                            ms.append(prs)
                            break
                        j=(j+1)%n
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        i = numpy.argmax(ms)
        if ms[i]>0:
            tspdtotal=tspdtotal-ms[i]
            if i//n==0:#makeflysaving
                label[tsp[i]]='drone'
                label[tsp[i-1]]='combined'
                label[tsp[(i+1)%n]]='combined'
                flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]])
                driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]])
                operation[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=max(flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])],driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])])
            elif i//n==1:#pushleftsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i-1)%n]]='truck'
                j=(i-2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[j-1]]=='combined':
                        flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i%n]])
                        driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=driveop[(tsp[dronenode],tsp[j-1],tsp[(i-1)%n])]+t_cost(V[tsp[(i-1)%n]],V[tsp[i%n]])
                        operation[(tsp[dronenode],tsp[j-1],tsp[i%n])]=max(flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])],driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])])
                        break
                    j=j-1
            elif i//n==2:#pushrightsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i+1)%n]]='truck'
                j=(i+2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[(j+1)%n]]=='combined':
                        flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=d_cost(V[tsp[i%n]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]])
                        driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i%n]],V[tsp[(i+1)%n]])
                        operation[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=max(flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])],driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])])
                        break
                    j=(j+1)%n
        else:
            for l in label:
                if label[l]=='simple':
                    label[l]='combined'

      
    return tspdtotal,label
            

In [8]:
def greedy_ph(alg,V,depot):
    tsp=alg(V)
    return greedy_core(tsp,depot)

In [13]:
greedy_ph(two_opt,V,0)

(135.6684277355113,
 {0: 'combined',
  6: 'drone',
  4: 'combined',
  1: 'drone',
  7: 'combined',
  8: 'drone',
  9: 'combined',
  3: 'combined',
  2: 'drone',
  5: 'combined'})

In [14]:
greedy_ph(mst,V,0)

(123.4987331590489,
 {0: 'combined',
  6: 'combined',
  4: 'drone',
  2: 'combined',
  3: 'combined',
  9: 'truck',
  8: 'drone',
  7: 'combined',
  1: 'drone',
  5: 'truck'})